## 기본 설정

### 환경변수 로드

In [ ]:
# pip install dotenv
import os
from dotenv import load_dotenv

load_dotenv()
HF_API_KEY = os.getenv("HF_API_KEY")
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


### Langchain 연결

In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com" # 이건 그대로
os.environ["LANGCHAIN_API_KEY"] = LANGSMITH_API_KEY # 이거 API키 받은거
os.environ["LANGCHAIN_PROJECT"] = "lm-studio-rag-tracing"  # 프로젝트 이름 설정(아무거나 해도됨)
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

### 기본 방법 선택하기

In [ ]:
# 청킹 방법
splitter_name = "semantic" # recursive, semantic

# LLM 모델
LLM_MODEL = "openai/gpt-oss-120b" # gpt-4o-mini, openai/gpt-oss-120b

# 벡터 DB
VECTOR_DB = "chroma_store" # chroma_store, faiss_index

# 임베딩 모델
# EB_MODEL = "text-embedding-3-large" # text-embedding-3-large -> 아래 임베딩 모델 섹션에서 설정



## 임베딩 모델 설정하기
- 임베딩 셀 중 한가지 선택해서 사용.

### OpenAI Embeddings

In [ ]:
# OpenAI Embeddings : text-embedding-3-large
# pip install langchain_openai
from langchain_openai import OpenAIEmbeddings

# 의미 기반으로 청킹을 하기 위해 OpenAI의 임베딩 모델을 사용
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
EB_MODEL = "text-embedding-3-large" # text-embedding-3-large
embedding = OpenAIEmbeddings(model=EB_MODEL, api_key=OPENAI_API_KEY)

### HuggingFaceEmbddings (local 저장)

In [ ]:
# 로컬에 저장해서 사용하는거라 api key가 필요없음
# HuggingFaceEmbeddings : sentence-transformers/stsb-xlm-r-multilingual

# pip install langchain sentence-transformers langchain-community
from langchain_community.embeddings import HuggingFaceEmbeddings

# HuggingFace 임베딩 클래스 이용
EB_MODEL = "stsb-xlm-r-multilingual" # stsb-xlm-r-multilingual
embedding = HuggingFaceEmbeddings(model_name=EB_MODEL)

# 단일 텍스트 임베딩 생성
sample_text = "금융권 문서 임베딩 예제"
embedding_vector = embedding.embed_query(sample_text)

print("임베딩 벡터 길이:", len(embedding_vector))
print("임베딩 벡터 일부:", embedding_vector[:5])


### Hugginface embeddings (multilinual-e5)

In [ ]:
# pip install langchain_huggingface
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings

EB_MODEL = "intfloat/multilingual-e5-large-instruct"  # multilingual-e5-large, multilingual-e5-large-instruct
model_name = EB_MODEL

HF_API_KEY = os.getenv("HF_API_KEY")

embedding = HuggingFaceEndpointEmbeddings(
    model=model_name,
    task="feature-extraction",
    huggingfacehub_api_token=HF_API_KEY,
)

# 단일 텍스트 임베딩 생성
sample_text = "금융권 문서 임베딩 예제"
embedding_vector = embedding.embed_query(sample_text)

print("임베딩 벡터 길이:", len(embedding_vector))
print("임베딩 벡터 일부:", embedding_vector[:5])

### Google Gemini Embeddings
- *호환성 문제로 genai 는 사용하지 않도록 한다.*

In [ ]:
# # pip install genai google google.genai
# from langchain.embeddings.base import Embeddings
# from google import genai
# from google.genai.types import EmbedContentConfig

# api_key = os.getenv("GC_API_KEY")

# client = genai.Client(api_key=api_key)
# EB_MODEL = "gemini-embedding-001"  # Google Cloud 임베딩 모델
# def call_google_cloud_embedding_api(text):
#     response = client.models.embed_content(
#         model=EB_MODEL,
#         contents=text,
#         config=EmbedContentConfig(
#             task_type="RETRIEVAL_DOCUMENT",
#             output_dimensionality=768
#         )
#     )
#     return response.embeddings[0].values

# class GoogleCloudEmbeddings(Embeddings):
#     def embed_documents(self, texts):
#         # Google Cloud 임베딩 API 호출 코드
#         # texts 리스트를 임베딩 벡터 리스트로 반환
#         embeddings = []
#         for text in texts:
#             embedding_vector = call_google_cloud_embedding_api(text)
#             embeddings.append(embedding_vector)
#         return embeddings

#     def embed_query(self, text):
#         return call_google_cloud_embedding_api(text)

# # 생성
# embedding = GoogleCloudEmbeddings()

# sample_text = "금융권 문서 임베딩 예제"
# embedding_vector = embedding.embed_query(sample_text)

# print("임베딩 벡터 길이:", len(embedding_vector))
# print("임베딩 벡터 일부:", embedding_vector[:5])


### KF-DeBERTa embedding 모델

In [ ]:
from langchain.embeddings.base import Embeddings
from transformers import AutoTokenizer, AutoModel
import torch

EB_MODEL = "kakaobank/kf-deberta-base"  # KF-DeBERTa 모델 이름
# KF-DeBERTa 임베딩 래퍼 정의
class KFDeBERTaEmbeddings(Embeddings):
    def __init__(self, model_name=EB_MODEL):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model.to(self.device)

    def embed_documents(self, texts):
        return [self.embed_query(text) for text in texts]

    def embed_query(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, max_length=512, padding=True)
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = self.model(**inputs)
            attention_mask = inputs["attention_mask"]
            token_embeddings = outputs.last_hidden_state
            input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
            embedding = torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        return embedding[0].cpu().numpy()

# 생성
embedding = KFDeBERTaEmbeddings()

sample_text = "금융권 문서 임베딩 예제"
embedding_vector = embedding.embed_query(sample_text)

print("임베딩 벡터 길이:", len(embedding_vector))
print("임베딩 벡터 일부:", embedding_vector[:5])

### bge-m3 Embedding model

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

EB_MODEL = "BAAI/bge-m3"
# 임베딩 모델 생성
embedding = HuggingFaceEmbeddings(
    model_name=EB_MODEL,
    model_kwargs={"device": "cpu"},  # GPU 사용 시
    encode_kwargs={"normalize_embeddings": True}
)

# sample_text = "금융권 문서 임베딩 예제"
# embedding_vector = embedding.embed_query(sample_text)

# print("임베딩 벡터 길이:", len(embedding_vector))
# print("임베딩 벡터 일부:", embedding_vector[:5])

## 청킹 방법 설정하기

### splitter 설정

In [ ]:
# pip install langchain_experimental
if splitter_name == "recursive" :
    # recursive_character_text_splitter로 구분자 재귀 청킹 기법
    from langchain_text_splitters import RecursiveCharacterTextSplitter

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        # separators=["\n\n", "\n", ".", "!", "?", " ", ""] # 이와 같이 separators를 지정할 수 있다.
    )
elif splitter_name == "semantic":
    # pip install langchain_experimental
    from langchain_experimental.text_splitter import SemanticChunker

    text_splitter = SemanticChunker(embedding)

print(text_splitter, "\nembedding: ", EB_MODEL, "splitter_name: ", splitter_name)

### Debug 함수

In [ ]:
def debug_chunkinfo_aftersplit(all_splits):
    for i, split in enumerate(all_splits):
        print(f"Chunk {i+1}:")
        print(split.page_content)
        print("-" * 40)

### 추출 하기

- pdf에서 추출

In [ ]:
import os
import glob
from langchain_community.document_loaders import PyPDFLoader

# PDF 파일을 읽어서 텍스트 데이터 추출 및 청킹
def extract_documents_from_pdf(pdf_path):
    # PDF 파일을 읽어서 텍스트 데이터 추출
    loader = PyPDFLoader(pdf_path)
    data_nyc = loader.load()

    # 추출된 텍스트 데이터를 청킹
    splits = text_splitter.split_documents(data_nyc)
    # debug_chunkinfo_aftersplit(splits) 

    # recursive_character_text_splitter의 경우, 청크가 겹치는 부분이 없으면 연결하지 않음
    # 따라서, 청크가 겹치는 부분이 없을 때는 직접 연결하여 오버랩을 만듦
    # 만약 청크가 겹치는 부분이 있다면, 그 부분은 자동으로 연결됨
    # 여기서는 청크가 겹치지 않는 경우에만 오버랩을 추가함

    # 만약 첫 번째 청크의 끝과 두 번째 청크의 시작이 겹치지 않는다면,
    if splitter_name == "recursive" and (len(splits) > 1) and (splits[0].page_content[-100:] == splits[1].page_content[:100]):
        print(splits[0].page_content)
        print("----")
        print(splits[1].page_content)
        for i in range(len(splits) - 1):
            splits[i].page_content += "\n" + splits[i + 1].page_content[:50]

    return splits


## 텍스트를 벡터로 변환하기

### 토큰 제한 실행 및 저장 함수 선언

- 토큰 제한하며 실행 및 저장

In [ ]:
# pip install pypdf
import tiktoken
import pypdf

try:
    encoding = tiktoken.encoding_for_model(EB_MODEL)
except KeyError:
    encoding = tiktoken.get_encoding("cl100k_base") # 토크나이저를 직접 지정


TOKEN_LIMIT_PER_BATCH = 39000  # 적절한 토큰 제한

def batch_save(vectorstore, splits):
    current_batch = []
    current_tokens = 0

    for doc in splits: 
        tokens = len(encoding.encode(doc.page_content))
        
        if current_tokens + tokens > TOKEN_LIMIT_PER_BATCH:
            try:
                vectorstore.add_documents(current_batch)
                print(f" {len(current_batch)}개 문서 배치 저장 완료")
            except Exception as e:
                print(f"배치 저장 중 오류 발생: {e}")
            current_batch = [doc]
            current_tokens = tokens
        else:
            current_batch.append(doc)
            current_tokens += tokens

    # 마지막 배치 처리
    if current_batch:
        vectorstore.add_documents(current_batch)
        print(f"마지막 배치 {len(current_batch)}개 문서 저장 완료")
    
    return vectorstore

### Vector DB 생성 혹은 불러오기 (Chroma, FAISS)

In [ ]:
# pip install faiss-cpu
import faiss
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.vectorstores import FAISS
from langchain.docstore.in_memory import InMemoryDocstore
from langchain_chroma import Chroma
import shutil
import time

persist_directory = './dbstore/' + VECTOR_DB + '_' + splitter_name + '_' + EB_MODEL.replace('/', '_')

if not os.path.exists(persist_directory):
    print(f"Creating new {VECTOR_DB} at {persist_directory}...")    

    # 만약 persist_directory가 존재하지 않는다면, 새로 생성. data에 있는 PDF 파일을 읽어서 청킹 후 저장
    current_dir = os.getcwd() # 현재 폴더 경로
    folder_path = os.path.join(current_dir, "data") # data 폴더 경로 설정
    pdf_files = glob.glob(os.path.join(folder_path, "*.pdf")) # PDF 파일 목록 가져오기

    all_splits = []
    print(f"현재 청킹 방법 : {splitter_name}")
    print(f"현재 임베딩 모델 : {EB_MODEL}")
    print(f"PDF 파일 개수: {len(pdf_files)}")
    for pdf_file in pdf_files:
        pdf_path = pdf_file
        # temp_docs, merged_path = extract_documents_from_pdf(pdf_file)
        print(f"Processing {pdf_path}...")
        all_splits.extend(extract_documents_from_pdf(pdf_file))
        time.sleep(2)  # PDF 파일 처리 간에 잠시 대기

    print(f"전체 청크 개수: {len(all_splits)}")

    if VECTOR_DB == "chroma_store":
        vectorstore = Chroma(
            embedding_function=embedding,
            persist_directory=persist_directory
        )
        vectorstore = batch_save(vectorstore, all_splits)
    elif VECTOR_DB == "faiss_index":
    
        embedding_dim = len(embedding.embed_query("test"))
        index = faiss.IndexFlatIP(embedding_dim)

        vectorstore = FAISS(
            embedding_function=embedding.embed_query,
            index=index,
            docstore=InMemoryDocstore(),
            index_to_docstore_id={}
        )
        vectorstore = batch_save(vectorstore, all_splits)
        vectorstore.save_local(persist_directory)
else :
    # 이미 존재하는 Chroma store를 불러오기
    print(f"Loading existing {VECTOR_DB}...")
    if VECTOR_DB == "faiss_index":
        vectorstore = FAISS.load_local(persist_directory, embedding, allow_dangerous_deserialization=True)
    elif VECTOR_DB == "chroma_store":
        vectorstore = Chroma(
            embedding_function=embedding,
            persist_directory=persist_directory
        )

    # 만약 새로 추가할 PDF 파일이 있다면 데이터 추출 및 저장후, extra_data 폴더에서 data 폴더로 이동
    src_folder = 'extra_data'
    dst_folder = 'data'

    current_dir = os.getcwd() # 현재 폴더 경로
    folder_path = os.path.join(current_dir, src_folder) # extra_data 폴더 경로 설정
    pdf_files = glob.glob(os.path.join(folder_path, "*.pdf")) # PDF 파일 목록 가져오기

    extra_splits = []
    if not pdf_files:
        print("추가할 PDF 파일이 없습니다.")
    else:
        print(f"추가할 PDF 파일 개수: {len(pdf_files)}")
        for pdf_file in pdf_files:
            print(f"Processing {pdf_file}...")
            extra_splits.extend(extract_documents_from_pdf(pdf_file))
            time.sleep(1)

        vectorstore = batch_save(vectorstore, extra_splits)
        if VECTOR_DB == "faiss_index":
            vectorstore.save_local(persist_directory)
        # extra_data 폴더의 모든 파일 중 .pdf만 이동
        for filename in os.listdir(src_folder):
            if filename.lower().endswith('.pdf'):
                src_path = os.path.join(src_folder, filename)
                dst_path = os.path.join(dst_folder, filename)
                if os.path.isfile(src_path):
                    shutil.move(src_path, dst_path)

# semantic text-embedding-3-large chroma 12m 23.7s 청크 개수 342
# semantic stsb-xlm-r-multilingual chroma 2m 38.5s 청크 개수 342
# semantic intfloat/multilingual-e5-large-instruct chroma 6m 29.6s 청크 개수 342
# semantic kakaobank/kf-deberta-base chroma 9m 53.6s 청크 개수 342
# semantic BAAI/bge-m3 chroma 31m 48.9s 청크 개수 342



# semantic text-embedding-3-large faiss_index 12m 21.9s 청크 개수 342
# semantic stsb-xlm-r-multilingual faiss_index 3m 35.2s 청크 개수 342
# semantic intfloat/multilingual-e5-large-instruct faiss_index 7m 54.1s 청크 개수 342
# semantic kakaobank/kf-deberta-base faiss_index 9m 53.4s 청크 개수 342
# semantic BAAI/bge-m3 faiss_index 25m 3.8s 청크 개수 342

- split 내용 확인

In [ ]:
print(extra_splits)

for i, split in enumerate(extra_splits):
    print(f"Split {i}: len : {len(split.page_content)} \n{split.page_content} ")
    print("\n")

## Vector DB에 들어있는 청킹 확인하기

In [ ]:
chunking_data = vectorstore.get()
print("청킹 방법 : ", splitter_name)
print("임베딩 모델 : ", EB_MODEL)
print("벡터 DB : ", VECTOR_DB)
for i, chunk in enumerate(chunking_data["documents"]):
    print(f"-----Result index {i}-----")
    print("Chunk:", chunk)



## LLM 설정 후 질문 및 답변

### gpt-4o-mini 로 설정

In [ ]:
LLM_MODEL = "gpt-4o-mini"
print(LLM_MODEL, "모델로 설정됨")

### gpt-oss-120b 로 설정

In [ ]:
LLM_MODEL = "gpt-oss-120b"
print(LLM_MODEL, "모델로 설정됨")

### 설정된 모델로 응답 프롬프트 설정

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import ChatOpenAI
# from langchain_openai import ChatOpenAI # 라고 했을 때는 module 'openai' has no attribute 'DefaultHttpxClient' 오류가
from openai import OpenAI

if LLM_MODEL == "gpt-4o-mini":
    chat = ChatOpenAI(model=LLM_MODEL)
elif LLM_MODEL == "gpt-oss-120b":
    HF_API_KEY = os.getenv("HF_API_KEY")
    chat = ChatOpenAI(
        model="openai/gpt-oss-120b",  # Hugging Face Router의 모델
        openai_api_key=HF_API_KEY,
        openai_api_base="https://router.huggingface.co/v1"  # base_url 대신 사용
    )

question_answering_promt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            # "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. you should user performed_context. You should consider all of special situation and general situation. If the 'perforemd_context' is null, you just say 'it is empyt'. Please write your answer in a markdown table format with the main points. Be sure to include all your source and page numbers like (3 ~ 10) in your answer. If you have over one source, you should include all of them. Answer in Korean. Also please write the keywords on user question that you think. \n#Example Format: \n(brief summary of the answer) \n (table) \n  (detailed answer to the question) \n**출처** \n- (file source) (page source and page number) (Please write the quoted text within 20 characters and follow it with ... )\n\n 키워드 : (keywords)\n #Context: {context}",
            "당신은 사용자를 위한 전문 대출 상담 챗봇입니다. "
            "당신의 역할은 일반 신용대출 및 부동산 관련 대출에 대해 정확하고 도움이 되는 상담을 제공하는 것입니다. "
            "사용자의 질문에 대해 반드시 벡터 데이터베이스에서 검색된 retrieved context(RAG) 기반으로만 답변해야 하며, "
            "retrieved context에 없는 정보를 추론하거나 임의로 만들어내지 마십시오. "
            "다음 규칙을 반드시 따르십시오:\n"
            "1. 답변은 retrieved context에 포함된 정보에 기반하여 작성합니다.\n"
            "2. retrieved context에 관련 정보가 없을 경우, \"해당 내용은 문서에 존재하지 않으므로 전문 상담사에게 연결해 드리겠습니다.\" 라고 답변합니다.\n"
            "3. 답변 작성 시 일반적인 상황뿐만 아니라 발생할 수 있는 특수 상황도 함께 고려하여 설명합니다.\n"
            "4. 답변은 다음 Markdown 표 형식으로 작성합니다:\n"
            "5. 답변할 수 있는 종류가 많다면 최대 5개까지 설명합니다.\n"
            "(간단 요약)\n"
            "(표)\n"
            "(질문에 대한 상세한 설명)\n"
            "**출처**\n"
            "- (문서명) (페이지 범위) “(20자 이하 인용문)…\"\n"
            "5. 마지막 줄에는 사용자 질문에서 핵심이 되는 키워드를 다음 형식으로 작성합니다: "
            "키워드 : (keyword1, keyword2, ...)\n"
            "#Context: {context}"
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_promt)

- meta data에 있는 정보는 llm이 알 수 없으므로 출처 filename을 포함시켜서 새로운 Document 타입을 생성

In [ ]:
import os
from langchain.schema import Document

# Document type인 docs가 context로 넘겨졌을 때 llm은 content에 있는 정보 기반으로 답을 한다.
# 따라서 meta data에 있는 정보는 llm이 알 수 없으므로 출처 filename을 포함시켜서 새로운 Document 타입을 생성한다.
def format_docs_with_source_as_documents(docs):
    new_docs = []
    for d in docs:
        filename = os.path.basename(d.metadata.get("source", ""))
        # 기존 page_content 뒤에 출처 붙이기
        new_content = f"{d.page_content}\n출처: {filename}"

        # 새 리스트 생성 (metadata 유지)
        new_docs.append(
            Document(page_content=new_content, metadata=d.metadata)
        )
    return new_docs



### 질문 입력 및 context 답변 생성

- 사용자 질문 입력 및 Vector DB를 위한 질문 재생성 (Rewrite)

In [ ]:

from openai import OpenAI
load_dotenv()
HF_API_KEY = os.getenv("HF_API_KEY")
client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=HF_API_KEY
)
question = "우리 부부의 연소득은 총 합 6800만원이야. 한 명은 군에 종사하고 하나는 직장인이라는 것을 참고해줘. 받을 수 있는 대출은 뭐가 있을까?"

summary_completion = client.chat.completions.create(
    model="openai/gpt-oss-120b",
    messages=[{
        "role": "user", 
        # "content": f"너는 Vector DB 전문가야. 주어진 질문을 vector db에 넣기 위해 키워드를 찾아서 질문으로 바꿔줘. 효과적이면 단어들로 구성해도 돼. 없는 내용은 절대 넣지마. 백틱도 안 넣어도돼. The question is : {question}"
        "content": f"너는 Vector DB 전문가야. 주어진 질문을 vector db에 넣기 위해 질문에서 키워드를 찾아서 단어로 파이썬 리스트화 해줘. 백틱도 안 넣어도돼. The question is : {question}"
        }],
)

question_for_vectordb = summary_completion.choices[0].message.content
print(question_for_vectordb)




### Reranking 미실행

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={'k': 10})
# k를 3개에서 10개로 변경 -> 7

docs = retriever.invoke(question_for_vectordb)
print(type(docs[0]))
# print(docs)
formatted_context = format_docs_with_source_as_documents(docs)


print(docs[0].page_content)
# for d in formatted_context:
#     # print("출처:", d.metadata["source"])
#     # print(d.metadata["source"])
#     print(d.page_content)
#     print("-" * 40) 

eb_model_safe = EB_MODEL.replace("/", "_")
vector_db_safe = VECTOR_DB.replace("/", "_")
llm_model_safe = LLM_MODEL.replace("/", "_")

RERANKING = ""
with open(f"./results/txtfile/{RERANKING}context_docs_{splitter_name}_{eb_model_safe}_{vector_db_safe}_{llm_model_safe}.txt", "w", encoding="utf-8") as f:
    for i, item in enumerate(formatted_context):
        f.write(f"Document {i + 1}:\n\n")
        f.write(str(item) + "\n\n")
        f.write("---------\n\n")

### Reranking 실행

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# ------------------------------
# 1) top 10 후보 가져오기
# ------------------------------
retriever = vectorstore.as_retriever(search_kwargs={'k': 40})
docs = retriever.invoke(question_for_vectordb)  # Document 리스트

doc_texts = [doc.page_content for doc in docs]

# ------------------------------
# 2) Dense similarity 계산 (OpenAIEmbeddings 사용)
# ------------------------------
# query 임베딩
query_emb = embedding.embed_query(question_for_vectordb)
# 문서 임베딩
doc_embs = embedding.embed_documents(doc_texts)
dense_sim = cosine_similarity([query_emb], doc_embs)[0]

# ------------------------------
# 3) Sparse similarity 계산 (TF-IDF)
# ------------------------------
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(doc_texts)
q_vec = vectorizer.transform([question_for_vectordb])
sparse_sim = cosine_similarity(q_vec, X)[0]

# ------------------------------
# 4) Hybrid 점수 계산
# ------------------------------
alpha = 0.3  # dense weight
beta = 0.7   # sparse weight
hybrid_score = alpha * dense_sim + beta * sparse_sim

# ------------------------------
# 5) 상위 5개 선택
# ------------------------------
ranked_idx = np.argsort(-hybrid_score)[:10]
ranked_docs = [docs[i] for i in ranked_idx]

# ------------------------------
# 6) formatted_context에 반영
# ------------------------------
formatted_context = format_docs_with_source_as_documents(ranked_docs)

for d in formatted_context:
    print("출처:", d.metadata["source"])
    print("-" * 40)

# ------------------------------
# 7) 파일로 저장
# ------------------------------
eb_model_safe = EB_MODEL.replace("/", "_")
vector_db_safe = VECTOR_DB.replace("/", "_")
llm_model_safe = LLM_MODEL.replace("/", "_")
RERANKING = "reranking_"
with open(f"./results/txtfile/{RERANKING}context_docs_{splitter_name}_{eb_model_safe}_{vector_db_safe}_{llm_model_safe}.txt", "w", encoding="utf-8") as f:
    for i, item in enumerate(formatted_context):
        f.write(f"Document {i + 1}:\n\n")
        f.write(str(item) + "\n\n")
        f.write("---------\n\n")

### 생성된 context로 LLM 답변하기

In [ ]:
from langchain.memory import ChatMessageHistory

chat_history = ChatMessageHistory()

chat_history.add_user_message(question)

answer = document_chain.invoke(
    {
        "messages": chat_history.messages,
        "context": formatted_context,
    }
)

chat_history.add_ai_message(answer)

print("|분류|종류|")
print("|---|---|")
print("|청킹 방법|", splitter_name, "|")
print("|임베딩 모델|", EB_MODEL, "|")
print("|벡터 DB|", VECTOR_DB, "|")
print("|LLM 모델|", LLM_MODEL, "|\n")

print("답변 : ")

print(answer)

print("\n")
print("-" * 40)

vector_db_safe = VECTOR_DB.replace("/", "_")
llm_model_safe = LLM_MODEL.replace("/", "_")
llm_model_safe = LLM_MODEL.replace("/", "_")
with open(f"./results/txtfile/{RERANKING}context_docs_{splitter_name}_{eb_model_safe}_{vector_db_safe}_{llm_model_safe}.txt", "a", encoding="utf-8") as f:
    f.write("\n###############################\n")
    f.write("\n##########  Results  ##########\n")
    f.write("\n###############################\n")
    f.write(answer + "\n")

### langchain 안걸리는 gpt-oss-120b

In [ ]:
from openai import OpenAI
from langchain_openai import ChatOpenAI

load_dotenv()
HF_API_KEY = os.getenv("HF_API_KEY")
client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=HF_API_KEY
)

# print(formatted_context)
summary_completion = client.chat.completions.create(
    model="openai/gpt-oss-120b",
    messages=[{
        "role": "user", 
        "content": f"You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. you should user performed_context. You should consider all of special situation and general situation. If the 'perforemd_context' is null, you just say 'it is empyt'. Please write your answer in a markdown table format with the main points. Be sure to include all your source and page numbers like (3 ~ 10) in your answer. If you have over one source, you should include all of them. Answer in Korean. Also please write the keywords on user question that you think. \n#Example Format: \n(brief summary of the answer) \n (table) \n  (detailed answer to the question) \n**출처** \n- (file source) (page source and page number) (Please write the quoted text within 20 characters and follow it with ... )\n\n 키워드 : (keywords)\n #Context: {formatted_context} the question is {question}",
        # "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. you should user performed_context. You should consider all of special situation and general situation. If the 'perforemd_context' is null, you just say 'it is empyt'. Please write your answer in a markdown table format with the main points. Be sure to include all your source and page numbers like (3 ~ 10) in your answer. If you have over one source, you should include all of them. Answer in Korean. Also please write the keywords on user question that you think. \n#Example Format: \n(brief summary of the answer) \n (table) \n  (detailed answer to the question) \n**출처** \n- (file source) (page source and page number) (Please write the quoted text within 20 characters and follow it with ... )\n\n 키워드 : (keywords)\n #Context: {formatted_context}. The question is : {question}"
        }],
)

print("|분류|종류|")
print("|---|---|")
print("|청킹 방법|", splitter_name, "|")
print("|임베딩 모델|", EB_MODEL, "|")
print("|벡터 DB|", VECTOR_DB, "|")
print("|LLM 모델|", "gpt-oss-120b", "|\n")

print("답변 : ")

print(summary_completion.choices[0].message.content)

print("\n")
print("-" * 40)

vector_db_safe = VECTOR_DB.replace("/", "_")
llm_model_safe = LLM_MODEL.replace("/", "_")

RERANKING = "reranking_"
with open(f"./results/txtfile/{RERANKING}context_docs_{splitter_name}_{eb_model_safe}_{vector_db_safe}_{llm_model_safe}.txt", "a", encoding="utf-8") as f:
    f.write("\n###############################\n")
    f.write("\n##########  Results  ##########\n")
    f.write("\n###############################\n")
    f.write(summary_completion.choices[0].message.content + "\n")

## context 사용 없이 질문을 받았을 경우

In [ ]:

from openai import OpenAI
from langchain_openai import ChatOpenAI

load_dotenv()
HF_API_KEY = os.getenv("HF_API_KEY")
client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=HF_API_KEY
)
# print(formatted_context)
summary_completion = client.chat.completions.create(
    model="openai/gpt-oss-120b",
    messages=[{
        "role": "user", 
        "content": f"You are an assistant for question-answering tasks. Please keep the answer under 500 characters. Please write your answer in a markdown table format with the main points.  Answer in Korean. \n#Example Format: \n(brief summary of the answer) \n (table) \n  (detailed answer to the question) \n**출처** \n- (file source) (page source and page number and correct full url) (Please write the quoted text within 20 characters and follow it with ... )\n The question is : {question}"
        }],
)
print("\n청킹 방법", "context 사용 없음")
print("임베딩 모델", "context 사용 없음")
print("LLM 모델", "openai/gpt-oss-120b")
print("벡터 DB", "사용 없음")

print("답변 : ")
print("-" * 40)
print(summary_completion.choices[0].message.content)

## 질의 확장 구현
- 챗본으로 만들 경우 첫번째 질문을 이어 받아서 두번째 질문도 답변을 해야하니 질의 확장에 대한 기능이다. (필수 x)

In [ ]:
from langchain_core.output_parsers import StrOutputParser

query_for_other = "3천만원 일때는?"

query_augmentation_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "기존의 대화 내용을 활용하여 사용자가 질문한 의도를 파악해서 한 문장의 명료한 질문으로 변환하라. 대명사나 이, 저, 그와 같은 표현을 명확한 명사로 표현하라. : \n\n{query}",
        ),
    ]
)
query_augmentation_chain = query_augmentation_prompt | chat | StrOutputParser()

augmented_query = query_augmentation_chain.invoke(
    {
        "messages": chat_history.messages,
        "query": query_for_other,
    }
)

print(augmented_query)

In [ ]:
docs_other = retriever.invoke(augmented_query)
formatted_context_other = format_docs_with_source_as_documents(docs_other)

chat_history.add_user_message(query_for_other)

answer = document_chain.invoke(
    {
        "messages": chat_history.messages,
        "context": formatted_context_other,
    }
)

chat_history.add_ai_message(answer)

print("질문:", query_for_other)
print("답변:")
print("-" * 40)
print(answer)

print("\n청킹 방법", splitter_name)
print("임베딩 모델", EB_MODEL)
print("LLM 모델", LLM_MODEL)